In [1]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier as lgb
import re
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import dacon_law_class as dlc
from dacon_law_class import SimpleOps as so
from sklearn.model_selection import GridSearchCV as GSCV
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import xgboost.sklearn as xgb
import warnings
warnings.filterwarnings('ignore')
import spacy
spacy.prefer_gpu()

from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

 ___________________________
|                           |
|======== YearDream ========|
|===========================|
|==== DLC Well Imported ====|
|===========================|
|========= BYJASON =========|
|________11th_Jun_23________|



In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')

In [ ]:
# train.head()
# test
# sample_submission

## BERT

@article{turc2019,
  title={Well-Read Students Learn Better: On the Importance of Pre-training Compact Models},
  author={Turc, Iulia and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina},
  journal={arXiv preprint arXiv:1908.08962v2 },
  year={2019}
}


In [3]:
train_facts = dlc.alpha_numeric_3_cols(train, 'first_party', 'second_party', 'facts')
test_facts = dlc.alpha_numeric_3_cols(test, 'first_party', 'second_party', 'facts')

In [4]:
train_facts = pd.DataFrame(train['facts'])
test_fact = pd.DataFrame(test['facts'])

In [5]:
train_all_done, test_all_done = dlc.rename_tokenized(train, test, 'first_party', 'second_party', 'facts')


Some weights of the model checkpoint at nlpaueb/bert-base-uncased-contracts were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not in

AssertionError: Torch not compiled with CUDA enabled

# 여기